# Dataset Collection

In [1]:
!hostname

k006-004-v7.hpcfund


In [2]:
import os

WORK_DIR = os.getenv('WORK')

DATA_FOLDER = os.path.join(WORK_DIR, "data")

CACHED_DATA_FOLDER = os.path.join(WORK_DIR, "cached_data")

# Salvamos o path do Cache par ao HuggingFace
os.environ['HF_HOME'] = CACHED_DATA_FOLDER

CACHED_DATA_FOLDER

'/work1/lgarcia/renneruan/cached_data'

In [ ]:
import transformers 
from datasets import load_dataset, concatenate_datasets, load_from_disk

from multiprocessing import cpu_count
from bs4 import BeautifulSoup, MarkupResemblesLocatorWarning
from ftfy import fix_text

import warnings

warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

print(transformers.__version__)


4.48.1


In [3]:
os.chdir(WORK_DIR)
print(os.getcwd())

/work1/lgarcia/renneruan


In [4]:
!ls

bert-env  cached_data  data  notebooks	requirements.txt


In [5]:
ls -l

total 4
drwxr-xr-x 1 renneruan lgarcia    0 Sep 23 09:58 bert-env/
drwxr-xr-x 1 renneruan lgarcia    0 Sep 23 22:06 cached_data/
drwxr-xr-x 1 renneruan lgarcia    0 Sep 23 22:42 data/
drwxr-xr-x 1 renneruan lgarcia    0 Sep 23 20:26 notebooks/
-rw-r--r-- 1 renneruan lgarcia 2665 Sep 23 09:58 requirements.txt


### Wikipedia Pages

In [7]:
wikipedia = load_dataset(
    "wikimedia/wikipedia", 
    "20231101.pt", 
    split="train", 
    num_proc=cpu_count(),
    cache_dir=CACHED_DATA_FOLDER
)

# Mantemos apenas coluna de Texto
wikipedia = wikipedia.remove_columns([col for col in wikipedia.column_names if col != "text"])  

print(wikipedia, '\n')
print(wikipedia[0]["text"][:100])

Dataset({
    features: ['text'],
    num_rows: 1112246
}) 

Astronomia é uma ciência natural que estuda corpos celestes (como estrelas, planetas, cometas, nebul


### BrWac

In [13]:
brwac = load_dataset(
    "dominguesm/brwac", 
    # data_dir="dataset-brwac",
    split="train", 
    num_proc=cpu_count(),
    cache_dir=CACHED_DATA_FOLDER,
    trust_remote_code=True
)

brwac = brwac.remove_columns([col for col in brwac.column_names if col != "text"])

print(wikipedia, '\n')

README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/38 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3530796 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/44 [00:00<?, ?it/s]

Dataset({
    features: ['text'],
    num_rows: 1112246
}) 



In [14]:
brwac[0]["text"]["paragraphs"]

[['Conteúdo recente'],
 ['ESPUMA MARROM CHAMADA "NINGUÉM MERECE"'],
 ['31 de Agosto de 2015, 7:07 , por paulo soavinski - | No one following this article yet.'],
 ['Visualizado 202 vezes'],
 ['JORNAL ELETRÔNICO DA ILHA DO MEL'],
 ['Uma espuma marrom escuro tem aparecido com frequência na Praia de Fora.',
  'Na faixa de areia ela aparece disseminada e não chama muito a atenção.',
  'No Buraco do Aipo, com muitas pedras, ela aparece concentrada.',
  'É fácil saber que esta espuma estranha está lá, quando venta.',
  'Pequenos algodões de espuma começam a flutuar no espaço, pertinho da Praia do Saquinho.',
  'Quem pode ajudar na coleta deste material, envio a laboratório renomado e pagamento de análises, favor entrar em contato com o site.']]

In [20]:
for paragraph in brwac[0]["text"]["paragraphs"]:
    for doc in paragraph:
        soup = BeautifulSoup(doc, 'html.parser')
        print( fix_text(soup.get_text())  )

Conteúdo recente
ESPUMA MARROM CHAMADA "NINGUÉM MERECE"
31 de Agosto de 2015, 7:07 , por paulo soavinski - | No one following this article yet.
Visualizado 202 vezes
JORNAL ELETRÔNICO DA ILHA DO MEL
Uma espuma marrom escuro tem aparecido com frequência na Praia de Fora.
Na faixa de areia ela aparece disseminada e não chama muito a atenção.
No Buraco do Aipo, com muitas pedras, ela aparece concentrada.
É fácil saber que esta espuma estranha está lá, quando venta.
Pequenos algodões de espuma começam a flutuar no espaço, pertinho da Praia do Saquinho.
Quem pode ajudar na coleta deste material, envio a laboratório renomado e pagamento de análises, favor entrar em contato com o site.


In [21]:
def paragraph2doc(example):
    list_doc = []

    for row in example["text"]:
        str_doc = ""
        for paragraph in row["paragraphs"]:
            for doc in paragraph:
                soup = BeautifulSoup(doc, 'html.parser') 

                str_doc += fix_text(soup.get_text()) + '\n'

        list_doc.append(str_doc)
    return {"text" : list_doc}

preprocessed_brwac = brwac.map(
    paragraph2doc,
    batched = True,
    remove_columns=["text"],
    num_proc = cpu_count(),
)

# preprocessed_brwac = preprocessed_brwac.rename_column("paragraphs", "text")
preprocessed_brwac

Map (num_proc=16):   0%|          | 0/3530796 [00:00<?, ? examples/s]

/tmp/ipykernel_389183/1676243496.py:8: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  soup = BeautifulSoup(doc, 'html.parser')
/tmp/ipykernel_389183/1676243496.py:8: MarkupResemblesLocatorWarning: The input passed in on

Dataset({
    features: ['text'],
    num_rows: 3530796
})

In [24]:
raw_datasets = concatenate_datasets([wikipedia, preprocessed_brwac])

In [25]:
raw_datasets

Dataset({
    features: ['text'],
    num_rows: 4643042
})

In [6]:
concatenated_save_path = os.path.join(DATA_FOLDER, 'concatenated_raw_datasets')

In [ ]:
raw_datasets.save_to_disk(concatenated_save_path)

Saving the dataset (0/41 shards):   0%|          | 0/4643042 [00:00<?, ? examples/s]

### C4 (Muito Grande)

In [ ]:
# c4 = load_dataset(  
#     "allenai/c4", 
#     "pt", 
#     split="train[:1%]", 
#     cache_dir=CACHED_DATA_FOLDER,
#     num_proc=cpu_count()
# )

# c4 = c4.remove_columns([col for col in c4.column_names if col != "text"])

# print(c4, '\n')
# print(c4[0]["text"][:1000])

### Oscar

In [ ]:
# oscar = load_dataset(
#     'oscar-corpus/community-oscar', 
#     data_files='data/2024-38/pt_meta/*.jsonl.zst', 
#     split='train',
#     streaming=True
# )

### C4 Legal

In [ ]:
# mc4legal = load_dataset(
#     "joelito/mc4_legal", 
#     "pt", 
#     split='train', 
#     trust_remote_code=True, 
#     num_proc=cpu_count())

# mc4legal = mc4legal.remove_columns([col for col in mc4legal.column_names if col != "text"])  # only keep the 'text' column

# print(mc4legal, '\n')

# print(mc4legal["text"][0][:1000])

### Multi Legal Pile (Muito Grande)

In [ ]:
# # multilegal has 17M, which is too much, use only 5% (0.86M)
# multilegal = load_dataset('joelniklaus/Multi_Legal_Pile', 
#                           "pt_caselaw", 
#                           split='train[:5%]', 
#                           trust_remote_code=True, 
#                           num_proc=cpu_count()
#                           )

# multilegal = multilegal.remove_columns([col for col in multilegal.column_names if col != "text"])  # only keep the 'text' column

# print(multilegal, '\n')

# print(multilegal["text"][0][:1000])

### LeNER-BR

In [ ]:
!wget https://github.com/peluz/lener-br/archive/refs/heads/master.zip

!unzip master.zip

In [ ]:
# Iterate files and convert the documents to a dataset object

raw_directory_path = "lener-br-master/leNER-Br/raw_text"
raw_lener = { "text" : [] }

for file in os.listdir(raw_directory_path):
    filename = os.fsdecode(file)

    if filename.endswith(".txt"):

        with open(f"{raw_directory_path}/{filename}", 'r') as f:
            raw_lener["text"].append( f.read() )

lener = Dataset.from_dict(raw_lener)

print(lener, '\n')

print(lener["text"][0][:1000])